In [ ]:
import gym
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tensorboardX import SummaryWriter

In [ ]:
class SchedulerEnv(gym.Env):

    def __init__(self):
        
        #starting parameters
        num_gps = 10
        num_slots = 15
        num_pre_booked = 75
        to_book = [4,2,3,1]
        num_to_book = len(to_book)
        agent_pos = [0,0]
        reward_decay = 0.95
        
        #set parameters for the day
        self.num_gps = num_gps
        self.num_slots = num_slots
        self.num_pre_booked = num_pre_booked
        self.to_book = to_book
        self.num_to_book = num_to_book
        self.diary_slots = num_gps*num_slots
        self.agent_pos = agent_pos
        self.reward_decay = reward_decay

        #set action space to move around the grid
        self.action_space = gym.spaces.Discrete(4) #up, down, left, right
        
        #set observation space 
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(self.num_slots, self.num_gps), dtype=np.int32)
   
    #creates daily diary for each gp, randomly populates prebooked appointments and resets parameters
    def reset(self):

        #creates zero filled dataframe with row per time slot and column per gp
        self.state = np.zeros((self.num_slots, self.num_gps),dtype=float)

        #randomly enters a 1 for each pre booked appointments
        pre_booked = self.num_pre_booked
        while pre_booked>0:
            pre_booked -= 1
            self.state[np.random.randint(self.num_slots), np.random.randint(self.num_gps)] = 1
            
        #randomly sets the agent start space
        self.agent_pos = [np.random.randint(self.num_slots), np.random.randint(self.num_gps)]

        #resets parameters for new episode
        self.done = False
        self.reward = 0
        self.appt_idx = 0
        self.decay_steps = 1
        
        print('starting state', self.state.sum(), self.state)

        return self.state
    
    #calculates new position of the agent based on the action
    def move_agent(self, action):

        #set boundaries for the grid
        max_row = env.num_slots - 1
        max_col = env.num_gps - 1

        #set new co-ordinates for the agent
        new_row = self.agent_pos[0]
        new_col = self.agent_pos[1]

        #calculate what the new position may be based on the action without going out the griid
        if action == 0:
            print('up')
            new_row = max(self.agent_pos[0] - 1, 0)
        if action == 1:
            print('down')
            new_row = min(self.agent_pos[0] + 1, max_row)
        if action == 2:
            print('left')
            new_col = max(self.agent_pos[1] - 1, 0)
        if action == 3:
            print('right')
            new_col = min(self.agent_pos[1] + 1, max_col)

        new_pos = [new_row, new_col]
        print('new pos', new_pos)

        return new_pos

    #checks if we can look to book appointment starting here
    def check_bookable(self):
        return self.state[self.agent_pos[0], self.agent_pos[1]] == 0.0
    
    #checks if the appointment fits
    def check_and_book(self):
        max_row = env.num_slots - 1

        #checks if the appointment fits
        cells_to_check = self.to_book[self.appt_idx]
        if cells_to_check==1:
            print('good to check for single')
            if self.state[self.agent_pos[0], self.agent_pos[1]] == 0:
                self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                self.appt_idx += 1
                self.decay_steps = 1
            else:
                print('already taken')
                self.decay_steps += 1
        if cells_to_check==2:
            #check we're not at the bottom of the grid
            if self.agent_pos[0]<max_row:
                #check the next cells is also 0.0
                print('good to check for double')
                if self.state[self.agent_pos[0], self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+1), self.agent_pos[1]] == 0:
                    self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+1), self.agent_pos[1]] = 1
                    self.appt_idx += 1
                    print('go ahead and book')
                    self.decay_steps = 1
                else:
                    print('already taken')
                    self.decay_steps += 1
            else:
                print('not for double')
                self.decay_steps += 1
        if cells_to_check==3:
            #check we're not at the bottom of the grid
            if self.agent_pos[0]+1<max_row:
                print('good to check for treble')
                if self.state[self.agent_pos[0], self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+1), self.agent_pos[1]] == 0 \
                 and self.state[(self.agent_pos[0]+2), self.agent_pos[1]] == 0:
                    self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+1), self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+2), self.agent_pos[1]] = 1
                    self.appt_idx += 1
                    print('go ahead and book')
                    self.decay_steps = 1
                else:
                    print('already taken')
                    self.decay_steps += 1
            else:
                print('not for treble')
                self.decay_steps += 1
        if cells_to_check==4:
            #check we're not at the bottom of the grid
            if self.agent_pos[0]+2<max_row:
                #check the next cells is also 0.0
                print('good for quad')
                if self.state[self.agent_pos[0], self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+1), self.agent_pos[1]] == 0 \
                 and self.state[(self.agent_pos[0]+2), self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+3), self.agent_pos[1]] == 0:
                    self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+1), self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+2), self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+3), self.agent_pos[1]] = 1
                    self.appt_idx += 1
                    print('go ahead and book')
                    self.decay_steps = 1
                else:
                    print('already taken')
                    self.decay_steps += 1
            else:
                print('not for quad')
                self.decay_steps += 1

        next_state = self.state

        return next_state

    def step(self, action):

        print('start step' , self.decay_steps)
        #get new positioin of agent based on action
        self.agent_pos = self.move_agent(action)
        print('trying to book', self.to_book, self.appt_idx)
        
        #check if it's possible to book then book
        if self.check_bookable():
            self.state = self.check_and_book()
            print('checked here')
        else:
            print('not bookable')
            self.decay_steps += 1
        
        #work out if episode complete
        if self.appt_idx == len(self.to_book):
            print('all booked')
            self.done = True
            
        #work out rewards
        self.reward = self.reward_decay**self.decay_steps
        
        print('step', self.decay_steps)
        print('end step')

        info = {}

        return self.state, self.reward, self.done, info

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, action_size):
        super(Model, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 128) 
        )

        self.actor = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, action_size)
        )

        self.critic = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        init_out = self.net(x)
        return self.actor(init_out), self.critic(init_out)

In [ ]:
#convert numpy array to tensor for input
def tensor_convert(x):
    return torch.from_numpy(x).float()

In [ ]:
#initialise environment, model and optimiser
env = SchedulerEnv()
model = Model((env.diary_slots), env.action_space.n)

#optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, 
#                             amsgrad=False)
#optimizer.zero_grad()

writer = SummaryWriter()

state = env.reset()
print('start pos', env.agent_pos)
done = False

while not done:

    #create model input from flattened grid
    nn_input = torch.flatten(tensor_convert(state))
    actor, critic = model(nn_input)
    
    print('get new action')
    #action = torch.argmax(actor)
    action = env.action_space.sample()

    #print(F.softmax(actor), action)

    #run through step to book appointment
    new_state, reward, done, info = env.step(action)
    state = new_state


In [ ]:
print(env.state.sum())

In [ ]:
env.appt_idx

In [ ]:
env.action_space.sample()